In [ ]:
import pandas as pd
import geopandas as gpd
import pandas as pd
import plotly.express as px
from pathlib import Path

In [ ]:
shape_path = Path("./../../data/counties/cb_2020_us_county_500k.shp")
data_path = Path("./../../data/industry_load/2014_update_20170910-0116.csv")

In [ ]:
gdf = gpd.read_file(shape_path)
df = pd.read_csv(data_path)

In [ ]:
county_2_state = df.set_index("fips_matching")["fipstate"].to_dict()
fips_2_state = {
    "01": "ALABAMA",
    "02": "ALASKA",
    "04": "ARIZONA",
    "05": "ARKANSAS",
    "06": "CALIFORNIA",
    "08": "COLORADO",
    "09": "CONNECTICUT",
    "10": "DELAWARE",
    "11": "DISTRICT OF COLUMBIA",
    "12": "FLORIDA",
    "13": "GEORGIA",
    "15": "HAWAII",
    "16": "IDAHO",
    "17": "ILLINOIS",
    "18": "INDIANA",
    "19": "IOWA",
    "20": "KANSAS",
    "21": "KENTUCKY",
    "22": "LOUISIANA",
    "23": "MAINE",
    "24": "MARYLAND",
    "25": "MASSACHUSETTS",
    "26": "MICHIGAN",
    "27": "MINNESOTA",
    "28": "MISSISSIPPI",
    "29": "MISSOURI",
    "30": "MONTANA",
    "31": "NEBRASKA",
    "32": "NEVADA",
    "33": "NEW HAMPSHIRE",
    "34": "NEW JERSEY",
    "35": "NEW MEXICO",
    "36": "NEW YORK",
    "37": "NORTH CAROLINA",
    "38": "NORTH DAKOTA",
    "39": "OHIO",
    "40": "OKLAHOMA",
    "41": "OREGON",
    "42": "PENNSYLVANIA",
    "44": "RHODE ISLAND",
    "45": "SOUTH CAROLINA",
    "46": "SOUTH DAKOTA",
    "47": "TENNESSEE",
    "48": "TEXAS",
    "49": "UTAH",
    "50": "VERMONT",
    "51": "VIRGINIA",
    "53": "WASHINGTON",
    "54": "WEST VIRGINIA",
    "55": "WISCONSIN",
    "56": "WYOMING",
}

In [ ]:
df.columns

In [ ]:
shapes = gdf[["GEOID", "geometry"]].set_index("GEOID")
shapes.index = shapes.index.astype(int)
shapes.head()

In [ ]:
energy = (
    df[["fips_matching", "Total"]]
    .rename(columns={"fips_matching": "GEOID"})
    .groupby("GEOID")
    .sum()
)
energy.index = energy.index.astype(int)
energy.head()

In [ ]:
final = shapes.join(energy).fillna(0)
final["state"] = final.index.map(county_2_state)
final = final.dropna()
final["state"] = final.state.map(lambda x: fips_2_state["{:02d}".format(int(x))])
final

In [ ]:
filtered = final[
    final.state.isin(
        [
            "CALIFORNIA",
            "WASHINGTON",
            "IDAHO",
            "OREGON",
            "NEW MEXICO",
            "NEVADA",
            "UTAH",
            "WYOMING",
            "MONTANA",
            "ARIZONA",
            "COLORADO",
        ]
    )
]
# filtered = final.copy()
px.choropleth(
    filtered,
    geojson=filtered.geometry,
    locations=filtered.index,
    color="Total",
    color_continuous_scale="Viridis",
    # range_color=(0, 12),
    scope="usa",
)